In [8]:
rdd = sc.textFile('/etc/hosts')
rdd.toDebugString()

'(2) /etc/hosts MapPartitionsRDD[9] at textFile at NativeMethodAccessorImpl.java:-2 []\n |  /etc/hosts HadoopRDD[8] at textFile at NativeMethodAccessorImpl.java:-2 []'

In [9]:
filetered_rdd = rdd.filter(lambda line: "localhost" in line)
filetered_rdd.toDebugString()

'(2) PythonRDD[10] at RDD at PythonRDD.scala:42 []\n |  /etc/hosts MapPartitionsRDD[9] at textFile at NativeMethodAccessorImpl.java:-2 []\n |  /etc/hosts HadoopRDD[8] at textFile at NativeMethodAccessorImpl.java:-2 []'

In [10]:
for line in filetered_rdd.collect():
    print line

# localhost is used to configure the loopback interface
127.0.0.1	localhost
::1             localhost 
fe80::1%lo0	localhost
